# Suspicious WMI execution
Detects WMI executing suspicious commands

## Rule Content
```
- title: Suspicious WMI execution
  id: 526be59f-a573-4eea-b5f7-f0973207634d
  status: experimental
  description: Detects WMI executing suspicious commands
  references:
  - https://digital-forensics.sans.org/blog/2010/06/04/wmic-draft/
  - https://www.hybrid-analysis.com/sample/4be06ecd234e2110bd615649fe4a6fa95403979acf889d7e45a78985eb50acf9?environmentId=1
  - https://blog.malwarebytes.com/threat-analysis/2016/04/rokku-ransomware/
  author: Michael Haag, Florian Roth, juju4
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image:
      - '*\wmic.exe'
      CommandLine:
      - '*/NODE:*process call create *'
      - '* path AntiVirusProduct get *'
      - '* path FirewallProduct get *'
      - '* shadowcopy delete *'
    condition: selection
  fields:
  - CommandLine
  - ParentCommandLine
  tags:
  - attack.execution
  - attack.t1047
  - car.2016-03-002
  falsepositives:
  - Will need to be tuned
  - If using Splunk, I recommend | stats count by Computer,CommandLine following for
    easy hunting by Computer/CommandLine.
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:(*\\wmic.exe) AND process_command_line.keyword:(*\/NODE\:*process\ call\ create\ * OR *\ path\ AntiVirusProduct\ get\ * OR *\ path\ FirewallProduct\ get\ * OR *\ shadowcopy\ delete\ *))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()